# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [33]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os

import pandas as pd
from google.cloud import bigquery
 

In [34]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="infinite-blade-452909-r4-d980a9c88b70.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [35]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2022-02-07,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,AR,2020-11-27,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
2,AR,2021-03-04,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
3,AR,2021-04-06,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
4,AR,2022-04-24,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
5,AR,2020-09-16,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
6,AR,2020-05-06,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
7,AR,2020-02-27,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
8,AR,2020-02-29,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
9,AR,2020-08-10,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)


# Część 3
Misja dodatkowa - Poznanie danych

### 3.1 Ile jest zapisanych wierszy z danymi.

In [36]:
dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

### 3.2 Ile krajów jest uwzględnionych w danych.

In [37]:
query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


np.int64(246)

 ### 3.3. W jaki sposób zapisywane są dzienne informacje dla krajów.

In [38]:
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [39]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


np.True_

### 3.4. W jaki sposób zapisywane są wartości liczbowe.

In [40]:
country_all.dtypes.value_counts()

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [41]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [42]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Jaki przedział czasowy jest uwzględniony w danych. Jakie są przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób.

In [43]:
query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level=0
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [44]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [45]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0 AND aggregation_level=0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0 AND aggregation_level=0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-13,new_confirmed
1,Afghanistan,2020-03-21,2022-09-12,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [46]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [47]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [48]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [49]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [50]:
query = ("""
SELECT DISTINCT date, country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
ORDER BY country_name
LIMIT 6
""")

df_temp= client.query(query).to_dataframe()
df_temp

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,Country_name,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-02-14,Afghanistan,AFG,0,0,0,0
1,2022-07-16,Afghanistan,AFG,0,2,1,0
2,2020-08-20,Afghanistan,AFG,2,2,0,0
3,2020-04-17,Afghanistan,AFG,3,3,2,4
4,2021-01-28,Afghanistan,AFG,1,0,0,0
5,2021-04-01,Afghanistan,AFG,0,2,2,0


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


### 4.1 Podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [51]:
query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data["GDP_2022"] = country_data["population"] * country_data["gdp_per_capita_usd"]
country_data.drop(columns=["gdp_per_capita_usd"], inplace=True)
country_data

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,GDP_2022
0,Colombia,COL,50882884,1141748,45.861,40827302,327278709888
1,Germany,DEU,83019213,357580,234.700,64324835,3947895655002
2,Spain,ESP,46937060,505935,93.100,37927409,1423413281560
3,Hong Kong,HKG,7496988,1110,7139.989,7507400,365515649940
4,Malaysia,MYS,32655400,330345,98.512,24475766,372728735600
...,...,...,...,...,...,...,...
241,Nicaragua,NIC,6624554,130370,55.049,3846137,12666147248
242,Niue,NIU,1618,260,6.223,<NA>,<NA>
243,Sierra Leone,SLE,7976985,72300,110.515,3319366,4020400440
244,Turkmenistan,TKM,6031187,488100,12.834,3092738,42013248642


### 4.2 statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [52]:
# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,CYM,0
1,2020-01-01,LAO,0
2,2020-01-01,RWA,0
3,2020-01-01,BOL,0
4,2020-01-01,ERI,0
...,...,...,...
243207,2022-09-15,BRA,9931
243215,2022-09-15,TWN,45405
243235,2022-09-15,MYS,2375
243287,2022-09-15,IDN,2651


### 4.3 Uwypuklenie problemu śmiertelności.

In [53]:
query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_deceased
0,2021-01-06,ESP,177
1,2022-05-14,FRA,74
2,2021-06-06,MYS,87
3,2021-12-15,PER,40
4,2022-02-07,ROU,81
...,...,...,...
243781,2022-03-14,PHL,15
243782,2020-11-03,SAU,15
243783,2021-07-29,COD,15
243784,2021-11-15,ZAF,15


### 4.4 Trendy i zależności dotyczące szczepień na COVID-19.

In [54]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vacinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vacinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_new_vacinations,New_persons_fully_vaccinated
0,2021-11-20,ABW,0,1
1,2021-06-10,ABW,257,300
2,2021-11-26,ABW,12,22
3,2021-05-06,BGD,43,102915
4,2022-08-29,BEL,16,17
...,...,...,...,...
243781,2021-03-14,BOL,431,0
243782,2021-12-17,MUS,0,0
243783,2021-05-14,PSE,0,0
243784,2022-05-17,PLW,0,0


### 4.5 Liczba wykonanych testów

In [55]:
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)

display(investment_in_vaccines)

C:\Users\Admin\miniconda3\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,date,3_letter_code,Total_tested
122,2020-01-01,ARG,4
189,2020-01-01,MEX,72
219,2020-01-01,BRA,1154
266,2020-01-02,MEX,89
475,2020-01-02,BRA,2337
...,...,...,...
243068,2022-09-15,ITA,146983
243137,2022-09-15,AUS,4367
243237,2022-09-15,AUT,67392
243264,2022-09-15,MYS,0


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [56]:
df_data_from_4 = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_deceased, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, covid_trend_analysis, on=['date', '3_letter_code'])
df_data_from_4 = pd.merge(df_data_from_4, investment_in_vaccines, on=['date', '3_letter_code'])
df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)
df_data_from_4

<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8772\397953275.py:5: SyntaxWarning: invalid escape sequence '\w'
  df_data_from_4.to_csv("..\wyniki\\data_from_4.csv", index=False)


,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Colombia,COL,50882884,1141748,45.861,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968
1,Colombia,COL,50882884,1141748,45.861,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720
2,Colombia,COL,50882884,1141748,45.861,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063
3,Colombia,COL,50882884,1141748,45.861,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016
4,Colombia,COL,50882884,1141748,45.861,40827302,327278709888,2021-03-18,6591,142,51364,774,43986
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,540542,300,1801.807,213645,5832448180,2022-03-08,325,0,12,36,1743
31436,Maldives,MDV,540542,300,1801.807,213645,5832448180,2022-03-09,172,0,12,32,2049
31437,Maldives,MDV,540542,300,1801.807,213645,5832448180,2022-03-10,204,0,10,51,1548
31438,Maldives,MDV,540542,300,1801.807,213645,5832448180,2022-03-12,196,0,33,29,1667


# Część 6


### 6.1 Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [57]:
df_data_from_4.drop('population', axis=1, inplace=True)
df_data_from_4.drop('area_sq_km', axis=1, inplace=True)
df_data_from_4.drop('population_density', axis=1, inplace=True)

df_data_from_4

,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986
...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667


### 6.2 Podstawowe infromacje geograficzne i demograficzne o krajach

In [58]:
# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1

world_countries = pd.read_csv("world_countries.csv")

df_with_countries = pd.merge(df_data_from_4, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
df_with_countries.drop('Country/Territory', axis=1, inplace=True)
df_with_countries.drop('CCA3', axis=1, inplace=True)

df_with_countries

,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986,...,47119728,44816108,39215135,32601393,26176195,20905254,1141748,45.4339,1.0069,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743,...,435582,361575,282507,224957,164887,123243,300,1745.9567,1.0045,0.01
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049,...,435582,361575,282507,224957,164887,123243,300,1745.9567,1.0045,0.01
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548,...,435582,361575,282507,224957,164887,123243,300,1745.9567,1.0045,0.01
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667,...,435582,361575,282507,224957,164887,123243,300,1745.9567,1.0045,0.01


In [59]:
df_gdp = pd.read_csv("gdp.csv")
df_gdp_pivot = df_gdp.pivot(index="Country Code", columns="Year", values="Value")
df_gdp_pivot.columns = [f"GDP_{int(year)}" for year in df_gdp_pivot.columns]
df_gdp_pivot.reset_index(inplace=True)
df_with_gdp = df_with_countries.merge(df_gdp_pivot, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_gdp.drop(columns=["Country Code"], inplace=True)
display(df_with_gdp)

,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,...,GDP_2007,GDP_2008,GDP_2009,GDP_2010,GDP_2011,GDP_2012,GDP_2013,GDP_2014,GDP_2015,GDP_2016
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986,...,2.074165e+11,2.439824e+11,2.338217e+11,2.870182e+11,3.354152e+11,3.696597e+11,3.801919e+11,3.781957e+11,2.915196e+11,2.824626e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743,...,1.868383e+09,2.271646e+09,2.345295e+09,2.588176e+09,2.774352e+09,2.886171e+09,3.295011e+09,3.697352e+09,4.006531e+09,4.224210e+09
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049,...,1.868383e+09,2.271646e+09,2.345295e+09,2.588176e+09,2.774352e+09,2.886171e+09,3.295011e+09,3.697352e+09,4.006531e+09,4.224210e+09
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548,...,1.868383e+09,2.271646e+09,2.345295e+09,2.588176e+09,2.774352e+09,2.886171e+09,3.295011e+09,3.697352e+09,4.006531e+09,4.224210e+09
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667,...,1.868383e+09,2.271646e+09,2.345295e+09,2.588176e+09,2.774352e+09,2.886171e+09,3.295011e+09,3.697352e+09,4.006531e+09,4.224210e+09


# Część 7

### 7.1 Dane związane z inflacją

In [60]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
df_inflation = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name="hcpi_a")

df_inflation.drop(columns=["IMF Country Code", "Indicator Type", "Series Name", "Note", "Country"], inplace=True)
df_inflation = df_inflation.melt(id_vars=["Country Code"], var_name="Year", value_name="Inflation")
df_inflation.dropna(subset=["Inflation"], inplace=True)
df_inflation = df_inflation.pivot(index="Country Code", columns="Year", values="Inflation")
df_inflation.columns = [f"Inflation_{int(year)}" for year in df_inflation.columns]
df_inflation.reset_index(inplace=True)
df_with_inflation = df_with_gdp.merge(df_inflation, left_on="3_letter_code", right_on="Country Code", how="left")
df_with_inflation.drop(columns=["Country Code"], inplace=True)
display(df_with_inflation)

,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,...,Inflation_2014,Inflation_2015,Inflation_2016,Inflation_2017,Inflation_2018,Inflation_2019,Inflation_2020,Inflation_2021,Inflation_2022,Inflation_2023
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968,...,2.897823,4.990232,7.51429,4.314,3.24,3.521,2.524621,3.495146,10.177240,11.735904
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720,...,2.897823,4.990232,7.51429,4.314,3.24,3.521,2.524621,3.495146,10.177240,11.735904
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063,...,2.897823,4.990232,7.51429,4.314,3.24,3.521,2.524621,3.495146,10.177240,11.735904
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016,...,2.897823,4.990232,7.51429,4.314,3.24,3.521,2.524621,3.495146,10.177240,11.735904
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986,...,2.897823,4.990232,7.51429,4.314,3.24,3.521,2.524621,3.495146,10.177240,11.735904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743,...,2.122000,1.891000,0.80100,2.272,1.37,1.309,-1.369774,0.543150,2.333143,3.156014
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049,...,2.122000,1.891000,0.80100,2.272,1.37,1.309,-1.369774,0.543150,2.333143,3.156014
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548,...,2.122000,1.891000,0.80100,2.272,1.37,1.309,-1.369774,0.543150,2.333143,3.156014
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667,...,2.122000,1.891000,0.80100,2.272,1.37,1.309,-1.369774,0.543150,2.333143,3.156014


### Dane o bezrobociu

In [61]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

df_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
df_unemployment.drop(columns=["Country Name", "Indicator Name", "Indicator Code", "Unnamed: 68"], inplace=True, errors="ignore")
df_unemployment = df_unemployment.melt(id_vars=["Country Code"], var_name="Year", value_name="Unemployment")
df_unemployment.dropna(subset=["Unemployment"], inplace=True)
df_unemployment = df_unemployment.pivot(index="Country Code", columns="Year", values="Unemployment")
df_unemployment.columns = [f"Unemployment_{int(year)}" for year in df_unemployment.columns]
df_unemployment.reset_index(inplace=True)
df_unemployment = df_with_inflation.merge(df_unemployment, left_on="3_letter_code", right_on="Country Code", how="left")
df_unemployment.drop(columns=["Country Code"], inplace=True)
display(df_unemployment)

,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,...,Unemployment_2014,Unemployment_2015,Unemployment_2016,Unemployment_2017,Unemployment_2018,Unemployment_2019,Unemployment_2020,Unemployment_2021,Unemployment_2022,Unemployment_2023
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968,...,8.799,8.572,8.922,9.086,9.360,10.280,15.983,13.898,10.547,9.594
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720,...,8.799,8.572,8.922,9.086,9.360,10.280,15.983,13.898,10.547,9.594
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063,...,8.799,8.572,8.922,9.086,9.360,10.280,15.983,13.898,10.547,9.594
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016,...,8.799,8.572,8.922,9.086,9.360,10.280,15.983,13.898,10.547,9.594
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986,...,8.799,8.572,8.922,9.086,9.360,10.280,15.983,13.898,10.547,9.594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743,...,7.477,6.604,5.771,5.401,4.950,4.562,5.434,5.024,4.419,4.241
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049,...,7.477,6.604,5.771,5.401,4.950,4.562,5.434,5.024,4.419,4.241
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548,...,7.477,6.604,5.771,5.401,4.950,4.562,5.434,5.024,4.419,4.241
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667,...,7.477,6.604,5.771,5.401,4.950,4.562,5.434,5.024,4.419,4.241


### Dane o przestępstwach w latach

In [62]:
df_crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")
df_crime = df_crime.iloc[:217]
df_crime.drop(columns=["Series Name", "Series Code", "Country Name"], inplace=True, errors="ignore")
df_crime.columns = df_crime.columns.str.replace(r"\s*\[YR\d+\]", "", regex=True)
df_crime = df_crime.melt(id_vars=["Country Code"], var_name="Year", value_name="Crime Rate")
df_crime["Year"] = df_crime["Year"].astype(int)
df_crime["Crime Rate"] = pd.to_numeric(df_crime["Crime Rate"], errors="coerce")
df_crime.dropna(subset=["Crime Rate"], inplace=True)
df_crime = df_crime.pivot(index="Country Code", columns="Year", values="Crime Rate")
df_crime.columns = [f"Crime_Rate_{int(year)}" for year in df_crime.columns]
df_crime.reset_index(inplace=True)
df_final = df_unemployment.merge(df_crime, left_on="3_letter_code", right_on="Country Code", how="left")
df_final.drop(columns=["Country Code"], inplace=True)
df_final.to_csv("..\wyniki\\final_data.csv",  index=False)
display(df_final)

<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:14: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8772\1459170941.py:14: SyntaxWarning: invalid escape sequence '\w'
  df_final.to_csv("..\wyniki\\final_data.csv",  index=False)


,Country_name,3_letter_code,population_urban,GDP_2022,date,Total_confirmed_cases,Total_new_deceased,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,...,Crime_Rate_2012,Crime_Rate_2013,Crime_Rate_2014,Crime_Rate_2015,Crime_Rate_2016,Crime_Rate_2017,Crime_Rate_2018,Crime_Rate_2019,Crime_Rate_2020,Crime_Rate_2021
0,Colombia,COL,40827302,327278709888,2021-03-14,2728,149,54597,6306,30968,...,35.908982,33.353569,28.585233,27.128764,26.040422,25.434488,26.231325,25.753473,24.242763,27.484365
1,Colombia,COL,40827302,327278709888,2021-03-15,4440,158,65338,5419,41720,...,35.908982,33.353569,28.585233,27.128764,26.040422,25.434488,26.231325,25.753473,24.242763,27.484365
2,Colombia,COL,40827302,327278709888,2021-03-16,5093,142,59933,2243,34063,...,35.908982,33.353569,28.585233,27.128764,26.040422,25.434488,26.231325,25.753473,24.242763,27.484365
3,Colombia,COL,40827302,327278709888,2021-03-17,5802,162,42313,5908,46016,...,35.908982,33.353569,28.585233,27.128764,26.040422,25.434488,26.231325,25.753473,24.242763,27.484365
4,Colombia,COL,40827302,327278709888,2021-03-18,6591,142,51364,774,43986,...,35.908982,33.353569,28.585233,27.128764,26.040422,25.434488,26.231325,25.753473,24.242763,27.484365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31435,Maldives,MDV,213645,5832448180,2022-03-08,325,0,12,36,1743,...,2.580382,0.748637,NaN,0.918312,3.962554,7.196651,NaN,0.594639,NaN,NaN
31436,Maldives,MDV,213645,5832448180,2022-03-09,172,0,12,32,2049,...,2.580382,0.748637,NaN,0.918312,3.962554,7.196651,NaN,0.594639,NaN,NaN
31437,Maldives,MDV,213645,5832448180,2022-03-10,204,0,10,51,1548,...,2.580382,0.748637,NaN,0.918312,3.962554,7.196651,NaN,0.594639,NaN,NaN
31438,Maldives,MDV,213645,5832448180,2022-03-12,196,0,33,29,1667,...,2.580382,0.748637,NaN,0.918312,3.962554,7.196651,NaN,0.594639,NaN,NaN
